In [140]:
from scipy.spatial import cKDTree
import pyfits, mechanize, bz2, os
from matplotlib import pyplot as pl
from numpy import *
from scipy.optimize import leastsq
import astropy.stats as sta
from scipy import ndimage
from glob import glob
import pandas as pd  #為了讀csv檔。csv檔是以逗點分割的檔案。
import requests
from bs4 import BeautifulSoup
from astropy.io import ascii
import concurrent.futures
from time import sleep
from astropy.table import Table
import math as m
import gzip
import patoolib

In [38]:
f = open('url.txt', 'r')
x = f.readlines()

In [193]:
x[0][0:-1]

'http://dr2.lamost.org/sas/fits/HD110609N070823M01/spec-56270-HD110609N070823M01_sp07-075.fits.gz'

In [53]:
len(x[5][0:-1])

91

In [81]:
x[5][0:-1][11:15]

'lamo'

In [128]:
for a in range(len(x[0][0:-1])):
    if x[0][0:-1][a:a+4] == 'spec':
        filename = x[0][0:-1][a:]

In [129]:
filename

'spec-56270-HD110609N070823M01_sp07-075.fits.gz'

In [132]:
filename[:-3]

'spec-56270-HD110609N070823M01_sp07-075.fits'

In [51]:
len(x)

52

In [90]:
for i in range(len(x)):
    
    url_lamost = x[i][0:-1]

    data_filename = url_lamost.split('/')[-1]
    
    
    for a in range(len(x[i][0:-1])):
        if x[i][0:-1][a:a+4] == 'spec':
            filename = x[i][0:-1][a:]
            if not os.path.exists(filename) :

                data = requests.get( url_lamost, stream=True)
                with open(data_filename, 'wb') as fdd:
                    for chunk in data.iter_content(chunk_size=1024): 
                        if chunk: # filter out keep-alive new chunks
                            fdd.write(chunk)

In [91]:
ptf1 = pyfits.open('spec-56270-HD110609N070823M01_sp07-075.fits')

In [107]:
ra = ptf1[0].header['RA']
dec = ptf1[0].header['Dec']

In [108]:
ra, dec

(167.929682, 5.417154)

In [109]:
all_target = pd.read_csv('epic_ha663_normal-hasub_new.csv')

In [113]:
for i in range(len(all_target['RA'])):
    if ra == all_target['RA'][i] and dec == all_target['Dec'][i]:
        print all_target['EPIC'][i]

201802777


In [115]:
for i in range(len(all_target['RA'])):
    if m.sqrt(((ra-all_target['RA'][i]))**2+((dec-all_target['Dec'][i]))**2) < 2./3600:
        print all_target['EPIC'][i]

201802777


In [124]:
os.chdir("D:\kepler_ptf_research\k2_c1\python_LAMOST_data_k2c1")

In [184]:
os.getcwd()

'D:\\kepler_ptf_research\\k2_c1\\python_LAMOST_data_k2c1'

In [122]:
f = gzip.open('spec-56270-HD110609N070823M01_sp07-075.fits.gz', 'rb')
file_content = f.read()
f.close()

In [226]:
#column_names = ['EPIC','RA','Dec','GZ','FITS']  

#df = pd.DataFrame([ ], columns = column_names)
#df.to_csv('lamost_data1.csv', index=False)

In [197]:
def append_csv(x1, x2, x3, x4, x5):

    df = pd.read_csv('lamost_data1.csv')
    column_names =  ['EPIC','RA','Dec','GZ','FITS']
    
    if x4 not in list(df['GZ'])  :
    
    
        df2 = pd.DataFrame([[x1, x2, x3, x4, x5]], columns = column_names)

        df_add = df.append(df2)

        df_add.to_csv('lamost_data1.csv', index = False)
    #else:
        #print x3,'already exists'

In [186]:
df = pd.read_csv('lamost_data1.csv')

def lamost_data_crawler(url_list):

    for i, v in enumerate(url_list):
        url_lamost = v[0:-1]

        data_filename = url_lamost.split('/')[-1]

        if data_filename not in list(df['GZ']) or data_filename not in list(df['PNG']):

            if not os.path.exists(data_filename) :

                data = requests.get( url_lamost, stream=True)
                with open(data_filename, 'wb') as fdd:
                    for chunk in data.iter_content(chunk_size=1024): 
                        if chunk: # filter out keep-alive new chunks
                            fdd.write(chunk)

            if '.gz' in data_filename and not os.path.exists(data_filename[:-3]):
                patoolib.extract_archive(data_filename)

                ptf1 = pyfits.open(data_filename[:-3])
                ra = ptf1[0].header['RA']
                dec = ptf1[0].header['Dec']
                
                for i in range(len(all_target['RA'])):
                    if m.sqrt(((ra-all_target['RA'][i]))**2+((dec-all_target['Dec'][i]))**2) < 2./3600:
                        
                        append_csv( all_target['EPIC'][i], all_target['RA'][i], all_target['Dec'][i], data_filename, data_filename[:-3])
                        
                        if not os.path.exists('ktwo'+str(all_target['EPIC'][i])+'_C1'):
                            os.mkdir('ktwo'+str( all_target['EPIC'][i])+'_C1')
                            
                        ptf1.close()
                        
                        if not os.path.exists('D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/ktwo'+str(all_target['EPIC'][i])+'_C1/'+data_filename[:-3]):
                            os.rename(data_filename[:-3], 'D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/ktwo'+str(all_target['EPIC'][i])+'_C1/'+data_filename[:-3])
                        if not os.path.exists('D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/ktwo'+str(all_target['EPIC'][i])+'_C1/'+filename):
                            os.rename(data_filename, 'D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/ktwo'+str(all_target['EPIC'][i])+'_C1/'+data_filename)


In [192]:
#lamost_data_crawler(x)

patool: Extracting spec-56283-HD111915N004324B01_sp04-083.fits.gz ...
patool: ... spec-56283-HD111915N004324B01_sp04-083.fits.gz extracted to `spec-56283-HD111915N004324B01_sp04-083.fits'.


WindowsError: [Error 183] 當檔案已存在時，無法建立該檔案。

In [136]:
if '.gz' in 'spec-56270-HD110609N070823M01_sp07-075.fits.gz':
    print 'yes'
else:
    print 'no'

yes


In [167]:
x[0][0:-1].split('/')[-1]

'spec-56270-HD110609N070823M01_sp07-075.fits.gz'

In [174]:
x_gz_list = []

for i in range(len(x)):
    if '.gz' in x[i]:
        x_gz_list.append(i)

In [175]:
len(x_gz_list)

26

In [224]:
df = pd.read_csv('lamost_data1.csv')

def lamost_data_crawler(url):
    

    url_lamost = url[0:-1]
    #url_lamost = url
    data_filename = url_lamost.split('/')[-1]
    #data_filename = url.split('/')[-1]
    
    if data_filename not in list(df['GZ']) and '.gz' in data_filename:

        if not os.path.exists(data_filename):

            data = requests.get( url_lamost, stream=True)
            with open(data_filename, 'wb') as fdd:
                for chunk in data.iter_content(chunk_size=1024): 
                    if chunk: # filter out keep-alive new chunks
                        fdd.write(chunk)

        if '.gz' in data_filename and not os.path.exists(data_filename[:-3]):
            patoolib.extract_archive(data_filename)

            ptf1 = pyfits.open(data_filename[:-3])
            ra = ptf1[0].header['RA']
            dec = ptf1[0].header['Dec']

            for i in range(len(all_target['RA'])):
                if m.sqrt(((ra-all_target['RA'][i]))**2+((dec-all_target['Dec'][i]))**2) < 2./3600:

                    append_csv( all_target['EPIC'][i], all_target['RA'][i], all_target['Dec'][i], data_filename, data_filename[:-3])

                    if not os.path.exists('ktwo'+str(all_target['EPIC'][i])+'_C1'):
                        os.mkdir('ktwo'+str( all_target['EPIC'][i])+'_C1')

                    ptf1.close()

                    if not os.path.exists('D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/ktwo'+str(all_target['EPIC'][i])+'_C1/'+data_filename[:-3]):
                        os.rename(data_filename[:-3], 'D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/ktwo'+str(all_target['EPIC'][i])+'_C1/'+data_filename[:-3])
                    if not os.path.exists('D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/ktwo'+str(all_target['EPIC'][i])+'_C1/'+filename):
                        os.rename(data_filename, 'D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/ktwo'+str(all_target['EPIC'][i])+'_C1/'+data_filename)
    
    elif '.png' in data_filename:
        if not os.path.exists(data_filename):

            data = requests.get( url_lamost, stream=True)
            with open(data_filename, 'wb') as fdd:
                for chunk in data.iter_content(chunk_size=1024): 
                    if chunk: # filter out keep-alive new chunks
                        fdd.write(chunk)
        
    
        

In [212]:
for i in range(len(x)):
    lamost_data_crawler(x[i])

patool: Extracting spec-56270-HD110609N070823M01_sp07-075.fits.gz ...
patool: ... spec-56270-HD110609N070823M01_sp07-075.fits.gz extracted to `spec-56270-HD110609N070823M01_sp07-075.fits'.
patool: Extracting spec-56283-HD111915N004324B01_sp04-083.fits.gz ...
patool: ... spec-56283-HD111915N004324B01_sp04-083.fits.gz extracted to `spec-56283-HD111915N004324B01_sp04-083.fits'.
patool: Extracting spec-56283-HD111915N004324B01_sp15-144.fits.gz ...
patool: ... spec-56283-HD111915N004324B01_sp15-144.fits.gz extracted to `spec-56283-HD111915N004324B01_sp15-144.fits'.
patool: Extracting spec-56283-HD111915N004324M01_sp09-084.fits.gz ...
patool: ... spec-56283-HD111915N004324M01_sp09-084.fits.gz extracted to `spec-56283-HD111915N004324M01_sp09-084.fits'.
patool: Extracting spec-56283-HD111915N004324M01_sp11-049.fits.gz ...
patool: ... spec-56283-HD111915N004324M01_sp11-049.fits.gz extracted to `spec-56283-HD111915N004324M01_sp11-049.fits'.
patool: Extracting spec-56283-HD111915N004324M01_sp11-0

In [225]:
for i in range(len(x)):
    lamost_data_crawler(x[i])

In [214]:
df = pd.read_csv('lamost_data1.csv')

In [221]:
if 'spec-56283-HD111915N004324B01_sp04-083.fits.gz' not in list(df['GZ']):
    print 'no'
else:
    print 'It is in list'


It is in list
